## Approach 3: fine-tune BERT for the Product Safety Dataset

build a fine-tune a BERT model to predict the safety of product.

In [ ]:
# Upgrade dependencies
!pip3 install -r ../../requirements.txt

In [ ]:
import boto3
import os
from os import path
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from transformers import Trainer, TrainingArguments, DistilBertForSequenceClassification, DistilBertTokenizerFast

## 1. Reading the dataset

This approach uses the __pandas__ library to read our dataset.

#### __Training data:__

In [ ]:
train_df = pd.read_csv('../../data/final_project/training.csv', encoding='utf-8', header=0)
train_df.head()

#### __Test data:__

In [ ]:
test_df = pd.read_csv('../../data/final_project/test.csv', encoding='utf-8', header=0)
test_df.head()

## 2. Train a fine-tune BERT Model

In [ ]:
# fill-in the missing values in it with the empty string
train_df["text"].fillna("", inplace=True)

In [ ]:
# Split train and validation data
train_texts, val_texts, train_labels, val_labels = train_test_split(
    test_df["text"].tolist(),
    test_df["title"].tolist(),
    test_size=0.10,
    shuffle=True,
    random_state=324,
)

In [ ]:
# Get tokenizer for BERT
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_texts,
                            truncation=True,
                            padding=True)
val_encodings = tokenizer(val_texts,
                          truncation=True,
                          padding=True)

In [ ]:
# Prepare Dataset for model
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
    
train_dataset = ReviewDataset(train_encodings, train_labels)
val_dataset = ReviewDataset(val_encodings, val_labels)

In [ ]:
# Initialize the model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",
                                                            num_labels=2)

In [ ]:
# function to calculate accuracy
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # calculate accuracy using sklearn's function
    acc = accuracy_score(labels, preds)
    return {
      'accuracy': acc
    }

training_args = TrainingArguments(
    output_dir="results",  # output directory
    num_train_epochs=10,  # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,  # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir="logs",  # directory for storing logs
    logging_steps=200,
    evaluation_strategy="steps",  # print val score at each step
    load_best_model_at_end=True
)

# Freeze the encoder weights until the classfier
for name, param in model.named_parameters():
    if "classifier" not in name:
        param.requires_grad = False

trainer = Trainer(
    model=model,  # the Transformers model
    args=training_args,  # training arguments
    train_dataset=train_dataset,  # passing training dataset
    eval_dataset=val_dataset,  # passing evaluation dataset
    compute_metrics=compute_metrics  # the callback that computes metrics of interest
)

trainer.train()

## 3. Make predictions on your test dataset

In [ ]:
# Prepare test dataset
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, n):
        self.encodings = encodings
        self.sz = n
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item
    def __len__(self):
        return self.sz
    
# fill-in the missing values in it with the empty string
test_df["text"] = test_df["text"].fillna("")

# Tokenize
test_encodings = tokenizer(test_df["text"].tolist(),
                            truncation=True,
                            padding=True)
# Get data set
test_dataset = TestDataset(test_encodings, len(test_df))

# Make prediction
test_prediction = trainer.predict(test_encodings)

## 4. Write your predictions to a CSV file

In [ ]:
result_df = pd.DataFrame()
result_df["ID"] = test_df["ID"]
result_df["human_tag"] = test_predictions
 
result_df.to_csv("../../data/approach_2_result.csv", encoding='utf-8', index=False)